In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [7]:
df = pd.read_csv("/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")

In [8]:
df['review'] = df['review'].str.lower()

# text preprcessing
import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text

df['review'] = df['review'].apply(remove_tags)


import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

def remove_stopwords(raw_text):
    cleaned_text = ' '.join([word for word in raw_text.split() if word not in stop])
    return cleaned_text

df['review'] = df['review'].apply(remove_stopwords)

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


In [9]:
df.head

<bound method NDFrame.head of                                                   review sentiment
0      one reviewers mentioned watching 1 oz episode ...  positive
1      wonderful little production. filming technique...  positive
2      thought wonderful way spend time hot summer we...  positive
3      basically there's family little boy (jake) thi...  negative
4      petter mattei's "love time money" visually stu...  positive
...                                                  ...       ...
49995  thought movie right good job. creative origina...  positive
49996  bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  catholic taught parochial elementary schools n...  negative
49998  i'm going disagree previous comment side malti...  negative
49999  one expects star trek movies high art, fans ex...  negative

[50000 rows x 2 columns]>

In [10]:
df = df[:10000]
df['sentiment'].value_counts()

sentiment
positive    5028
negative    4972
Name: count, dtype: int64

In [11]:
df.drop_duplicates(inplace=True)

/tmp/ipykernel_33/3006716147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [12]:
X = df.iloc[:,0:1]
y = df['sentiment']

In [13]:
#LabelEncoding the sentiments 
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(y)
y

array([1, 1, 1, ..., 0, 0, 1])

In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state=1)

## Vectorization

### Bag of words is done using countvectorizer library 

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [16]:
X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()

## Training of Model

In [17]:
from sklearn.naive_bayes import GaussianNB
gnb =  GaussianNB()
gnb.fit(X_train_bow,y_train)
y_pred = gnb.predict(X_test_bow)
from sklearn.metrics import accuracy_score,confusion_matrix
accuracy_score(y_test,y_pred)

0.6324486730095142

In [18]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report

In [19]:
# Define the parameter grid
param_grid = [
    {
        'classifier': [LogisticRegression()],
        'classifier__penalty': ['l1', 'l2'],
        'classifier__C': [0.1, 1, 10],
        'classifier__solver': ['liblinear']
    },
    {
        'classifier': [SVC()],
        'classifier__kernel': ['linear', 'rbf'],
        'classifier__C': [0.1, 1, 10],
        'classifier__gamma': [0.001, 0.01, 0.1, 1]
    },
    {
        'classifier': [RandomForestClassifier()],
        'classifier__n_estimators': [10, 50, 100],
        'classifier__max_features': ['auto', 'sqrt', 'log2'],
        'classifier__max_depth': [None, 10, 20, 30]
    },
    {
        'classifier': [GaussianNB()],
        'classifier__var_smoothing': np.logspace(0, -9, num=100)
    }
]


#### Using pipeline it is much more flexible and cleaner to do hyperparameter tuning.

#### Other methods
#### 1) Custom estimators 
#### 2) Using Functions

In [20]:
pipeline = Pipeline([
    ('classifier',LogisticRegression())
])

In [ ]:
grid_search = GridSearchCV(pipeline,param_grid,cv=5,scoring='accuracy',n_jobs=1)
grid_search.fit(X_train_bow, y_train)

In [ ]:
# Print the best parameters and best cross-validation score
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation accuracy: {grid_search.best_score_}")

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

print(f"Test accuracy: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

## Using Tfldf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf  = TfidfVectorizer()

X_train_tfidf = tfidf.fit_transform(X_train['review']).toarray()
X_test_tfidf = tfidf.transform(X_test['review']).toarray()

In [ ]:
rf = RandomForestClassifier()

rf.fit(X_train_tfidf,y_train)
y_pred = rf.predict(X_test_tfidf)

accuracy_score(y_test,y_pred)